# Nabson Paiva#

### Critérios de parada ###

Antes de implementarmos os métodos iterativos, temos que implementar um método para o critério de parada. No nosso caso, usaremos a _norma infinita_. A norma infinita é dada por:

$$\lVert v \rVert_{\infty} = \max_{1\leq i\leq n}|v_i|$$ 

E a distância entre duas soluções iteradas $x^k$ e $x^{k-1}$ pode ser dada por:

$$ \frac{\lVert x^k - x^{k-1} \rVert_{\infty}}{\lVert x^k\rVert_{\infty}} = \frac{\max\limits_{1\leq i\leq n}|x_i^{k} - x_i^{k-1}|}{\max\limits_{1\leq i\leq n}|x_i^k|} \leq \epsilon$$

Faça uma função para retornar a distância usando a norma infinita de dois vetores:

In [1]:
import numpy as np

In [2]:
def normaInfinita(v1,v2):
    return (np.abs(v2-v1)).max()/np.abs(v2).max()

In [3]:
A = np.array([[10,3,-2],[2,8,-1],[1,1,5]],dtype='float64')
b = np.array([57,20,-4],dtype='float64')
# x ~= [5.0, 1.0 -2.0]

## Jacobi ##

Para resolver o método de Jacobi, se deve decompor a matriz $A$ em três matrizes compostas dos seus elementos, $D$ (Diagonal), $E$ (triangular inferior) e $F$ (triangular superior).

De posse destes elementos, deve-se utilizar esta fórmula de iteração:

$$ x^{k+1}=-D^{-1}(E+F)x+D^{-1}b $$

Lembrando que para D ter inversa, nenhum elemento da diagonal principal de A pode ser nulo.

Desta forma, eu posso utilizar a forma matricial acima para computar as respostas. Implemente o método de Jacobi matricial abaixo:

In [4]:
def jacobim(A,b,niter=100,minimo = 0.000001):
    Di = np.zeros(A.shape)
    F = np.zeros(A.shape)
    E = np.zeros(A.shape)
    x0 = np.zeros(len(b))
    x = np.zeros(len(b))
    for i in range(len(A)):
        Di[i][i] = 1.0/A[i][i]
        F[i,i+1:] = A[i,i+1:]
        E[i,:i] = A[i,:i]
    J = -Di.dot(E+F)
    K = Di.dot(b)
    for i in range(niter):
        x = J.dot(x0) + K
        if (normaInfinita(x0,x) <= minimo):
            return x
        x0 = x.copy()
    return x

In [5]:
jacobim(A,b)

array([ 4.99999964,  1.00000022, -1.99999963])

Outra forma de implementar Jacobi é através da fórmula escalar:

$$x_i^{k+1}=\frac{1}{a_{ii}} \left( \sum\limits_{\substack{j=1 \\ i\neq j}}^{n}{a_{ij}x_j^k+b_i} \right)$$

Implemente a forma escalar do jacobi:

In [6]:
def jacobie(A,b,niter=1000,minimo = 0.000001):
    t = len(b)
    x0 = np.zeros(t,dtype='float64')
    x = np.zeros(t,dtype='float64')
    for n in range(niter):
        for i in range(t):
            x[i] = 1.0/A[i,i] * (-sum([A[i,j]*x0[j] for j in range(t) if i != j]) + b[i])
        if (normaInfinita(x0,x) <= minimo):
            return x
        x0 = x.copy()
    return x

In [7]:
jacobie(A,b)

array([ 4.99999964,  1.00000022, -1.99999963])

Para o método de Jacobi (e o Gauss-Seidel) convergirem para um resultado correto, elas tem que ter condições de convergência:

- Condição suficiente: Elementos da diagonal principal estritamente dominantes.

$$ |a_{ii}|> \sum\limits_{\substack{j=1 \\ j\neq i}}^{n}{|a_{ij}|,i=1,2,...,n}$$

Esta condição se verdadeira garante a convergência. Contudo, mesmo se ela for falsa o sistema ainda pode convergir.

- Condição necessária: $\rho(M)<1$, sendo $\rho(M)$ o raio espectral, o maior autovalor em módulo.

Faça uma função que verifique se um sistema com uma matriz de coeficientes A pode ser solucionado via jacobi, testando primeiro a condição suficiente e, em caso negativo, testando a condição necessária:

In [8]:
def podeJacobi(A):
    pode = True
    t = len(A)
    for i in range(t):
        if np.abs(A[i,i]) <= np.abs(sum([A[i,j] for j in range(t) if i != j])):
            pode = False
            break
    if (pode or np.abs(np.linalg.eig(A)[0]).max() < 1):
        pode = True
    return pode

In [9]:
podeJacobi(A)

True

## Gauss-Seidel ##

Função de iteração:

$$ x^{k+1}=D^{-1}(-Ex^{k+1}-Fx^{k}+b)$$

Lembrando que multiplicar por $D^{-1}$ é o mesmo que dividir cada linha pelo elemento da diagonal principal desta linha em A

A forma mais simples de fazer o Gauss Seidel matricialmente é linha a linha (calculando a formula acima pra cada linha da solução X)

Implemente Gauss-Seidel na forma Matricial e na forma escalar (acessando os elementos individualmente das matrizes)

Fórmula escalar:

$$ x^{k+1}_{i} = \frac{1}{a_{ii}}(-\sum\limits_{\substack{j=0}}^{i-1}{a_{ij}x^{k+1}_{j}} - \sum\limits_{\substack{j=i+1}}^{n}{a_{ij}x^{k}_{j} + b_{i}}), i=1,2,...,n$$

In [10]:
def gaussM(A,b,niter=1000,minimo = 0.000001):
    Di = np.zeros(A.shape)
    F = np.zeros(A.shape)
    E = np.zeros(A.shape)
    x0 = np.zeros(len(b))
    x = np.zeros(len(b))
    for i in range(len(A)):
        Di[i][i] = 1.0/A[i][i]
        F[i,i+1:] = A[i,i+1:]
        E[i,:i] = A[i,:i]
    for i in range(niter):
        x = Di.dot(-E.dot(x) - F.dot(x0) + b)
        if (normaInfinita(x0,x) <= minimo):
            return x
        x0 = x.copy()
    return x

def gaussE(A,b,niter=1000,minimo = 0.000001):
    t = len(b)
    x0 = np.zeros(t,dtype='float64')
    x = np.zeros(t,dtype='float64')
    for n in range(niter):
        for i in range(t):
            x[i] = 1.0/A[i,i]*(-sum([A[i,j]*x[j] for j in range(i)])-sum([A[i,j]*x0[j] for j in range(i+1,t)])+b[i])
        if (normaInfinita(x0,x) <= minimo):
            return x
        x0 = x.copy()
    return x

In [11]:
gaussM(A,b)

array([ 4.99999964,  1.00000022, -1.99999963])

In [12]:
gaussE(A,b)

array([ 5.00000015,  1.00000002, -2.00000003])

## Exercicios ##

### 1 - Tente resolver todos os sistemas (de M1 a M5) com Jacobi e Gauss-Seidel, tanto na forma matricial quanto na escalar e, nos casos em que o resultado _divergir_, verifique a condição de convergência dos métodos.

In [13]:
def resolveIterativo(M,b):
    if podeJacobi(M):
        print("Resultado converge por Jacobi e Gauss-Seidel.")
    else:
        print("Resultado diverge por Jacobi e Gauss-Seidel.")
    print("Solução por Jacobi Matricial:",jacobim(M,b))
    print("Solução por Jacobi Escalar:",jacobie(M,b))
    print("Solução por Gauss-Seidel Matricial:",gaussE(M,b))
    print("Solução por Gauss-Seidel Escalar:",gaussE(M,b))

In [14]:
M1 = np.array([[1,-3,5,6], [-8,4,-1,0],[3,2,-2,7],[1,2,5,-4]],dtype='float')
b1 = np.array([17,29,-11,7],dtype='float')
print("Sistema 1")
resolveIterativo(M1,b1)

M2 = np.array([[-2,3,1,5],[5,1,-1,0],[1,6,3,-1],[4,5,2,8]],dtype='float')
b2 = np.array([2,-1,0,6],dtype='float')
print("\nSistema 2")
resolveIterativo(M2,b2)

M3 = np.array([[9,-6,3],[-6,29,-7],[3,-7,18]],dtype='float')
b3 = np.array([-3,-8,33],dtype='float')
print("\nSistema 3")
resolveIterativo(M3,b3)

M4 = np.array([[4,-2,4,10],[-2,2,-1,-7],[4,-1,14,11],[10,-7,11,31]],dtype='float')
b4 = np.array([2,2,-1,-2],dtype='float')
print("\nSistema 4")
resolveIterativo(M4,b4)

M5 = np.array([[16,-4,4,12],[-4,2,-1,-7],[4,-1,26,13],[12,-7,13,25]],dtype='float')
b5 = np.array([2,2,-1,-2],dtype='float')
print("\nSistema 5")
resolveIterativo(M5,b5)

Sistema 1
Resultado diverge por Jacobi e Gauss-Seidel.
Solução por Jacobi Matricial: [1.25152030e+54 2.13425771e+54 1.87679433e+54 6.93490209e+53]
Solução por Jacobi Escalar: [nan nan nan nan]
Solução por Gauss-Seidel Matricial: [nan nan nan nan]
Solução por Gauss-Seidel Escalar: [nan nan nan nan]

Sistema 2
Resultado diverge por Jacobi e Gauss-Seidel.


/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in subtract
  
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


Solução por Jacobi Matricial: [1.55963841e+52 5.90446286e+52 3.11818759e+51 1.85883743e+51]
Solução por Jacobi Escalar: [nan nan nan nan]
Solução por Gauss-Seidel Matricial: [nan nan nan nan]
Solução por Gauss-Seidel Escalar: [nan nan nan nan]

Sistema 3
Resultado converge por Jacobi e Gauss-Seidel.
Solução por Jacobi Matricial: [-1.00000058e+00  3.45632528e-07  1.99999962e+00]
Solução por Jacobi Escalar: [-1.00000058e+00  3.45632528e-07  1.99999962e+00]
Solução por Gauss-Seidel Matricial: [-9.99999866e-01 -1.08684894e-08  1.99999997e+00]
Solução por Gauss-Seidel Escalar: [-9.99999866e-01 -1.08684894e-08  1.99999997e+00]

Sistema 4
Resultado diverge por Jacobi e Gauss-Seidel.
Solução por Jacobi Matricial: [ 2.80411320e+27 -3.61437233e+27  9.76323471e+26  1.06456711e+27]
Solução por Jacobi Escalar: [ 6.67871181e+286 -8.60855090e+286  2.32536372e+286  2.53553848e+286]
Solução por Gauss-Seidel Matricial: [ 2.00004163  5.99986782 -0.99997997  0.99994962]
Solução por Gauss-Seidel Escalar: [

### 2 - Use todos os métodos (Gauss, LU com pivotação, Choleski, Jacobi e Gauss-Seidel) e marquem o tempo com `%timeit -n1` para o seguinte sistema:

In [15]:
def resolveTS(A,b): #A Triangular Superior
    x = np.zeros(len(A))
    for i in range(len(A)-1,-1,-1):
        x[i] = (b[i] - (A[i][i+1:]*x[i+1:]).sum())/A[i][i]
    return x

def resolveTI(A,b): #A Triangular Inferior
    x = np.zeros(len(b))
    for i in range(len(b)):
        x[i] = (b[i] - (A[i][:i]*x[:i]).sum())/A[i][i] # x[<intervalo_fechado>:<intervalo_aberto>]
    return x

# Eliminação Gaussiana
def eliminacaoGaussianaSimples(A0,b):
    M = np.zeros(A0.shape)
    A = np.concatenate((A0,b.reshape(len(b),1)),axis=1)
    for i in range(len(A)-1):
        A[i+1:] -= (A[i+1:,i]/A[i][i]).reshape((len(M[i+1:]),1))*A[i]
    return resolveTS(A[:,:len(b)],A[:,len(b)])

# Decomposição LU com pivotação parcial
def LUparcial(A):
    t = A.shape[0]
    P = np.identity(t)
    U = A.copy()
    L = np.zeros(A.shape)
    for i in range(t):
        lp = np.argmax(np.abs(U[i:,i])) + i
        U[[i,lp]] = U[[lp,i]]
        L[[i,lp]] = L[[lp,i]]
        P[[i,lp]] = P[[lp,i]]
        L[i][i]=1
        for j in range(i+1,t):
            u = U[j][i]/U[i][i]
            L[j][i] = u
            U[j] -= U[i]*u
    return L,U,P

def resolveLUpar(L,U,P,b):
    y = resolveTI(L,np.dot(P,b))
    return resolveTS(U,y)

def LUpar(A,b):
    L,U,P = LUparcial(A)
    return resolveLUpar(L,U,P,b)

# Método de Cholesky
def verificaCholesky(A):
    return (A == A.T).all() and (A.diagonal() > 0).all() and (np.linalg.eig(A)[0] > 0).all() and verificaPivot(A)

def geraCholesky(A):
    L = np.zeros(A.shape)
    for j in range(len(A)):
        L[j][j] = np.sqrt(A[j][j] - (np.power(L[j][:j],2).sum()))
        for i in range(len(A)):
            L[i,j] = (A[i,j]- (L[i,:j]*L[j,:j]).sum())/L[j,j]
    j = len(A)-1
    L[j,j] = np.sqrt(A[j,j]-(L[j][:j]**2).sum())
    return L

def resolveLU(L,U,b):
    y = resolveTI(L,b)
    return resolveTS(U,y)

def cholesky(A,b):
    L = geraCholesky(A)
    return resolveLU(L,L.T,b)

In [16]:
# Estou fazendo 100x100 porque é o máximo que meu PC é capaz de processar
n = 100
MF = np.random.randint(1,20,(n,n))
bF = np.random.randint(-1000,1000,n)
MF = MF + 0.0
bF = bF + 0.0
print(MF)

[[ 9.  8. 10. ... 10. 18.  2.]
 [11.  3.  9. ... 11.  6. 11.]
 [ 7. 12. 15. ... 13.  7. 13.]
 ...
 [10. 16.  1. ... 13.  6.  3.]
 [ 8.  1.  9. ... 13. 11. 15.]
 [ 3. 16. 14. ...  9.  1.  6.]]


In [17]:
print("Resolução por Eliminação Gaussiana:")
%timeit -n1 eliminacaoGaussianaSimples(MF,bF)
print("Resolução por Decomposição LU Parcial:")
%timeit -n1 LUpar(MF,bF)
print("Resolução por Cholesky:")
%timeit -n1 cholesky(MF,bF)
print("Resolução por Jacobi:")
%timeit -n1 jacobie(MF,bF)
print("Resolução por Gauss-Seidel:")
%timeit -n1 gaussE(MF,bF)

Resolução por Eliminação Gaussiana:
4.96 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Resolução por Decomposição LU Parcial:
48.8 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Resolução por Cholesky:


/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in sqrt


70.6 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Resolução por Jacobi:


/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


6.2 s ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Resolução por Gauss-Seidel:


/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars
/home/nabson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


5.88 s ± 109 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
